In [2]:
import nltk
import os
import codecs
import argparse
import numpy as np
import string
import re
from pyvi import ViTokenizer
import gensim
from gensim.models.callbacks import CallbackAny2Vec
import pickle as pickle


pattern = r'[' + string.punctuation +']'

In [3]:
pattern

'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'

In [4]:
epoch_losses = []

In [5]:
class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0
        self.losses = []

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.epoch += 1
        self.losses.append(loss_now)
        epoch_losses.append(loss_now)
def get_min_count(sents):
    
    global vocab_size
    from itertools import chain
     
    fdist = nltk.FreqDist(chain.from_iterable(sents))
    min_count = fdist.most_common(vocab_size)[-1][1] # the count of the the top-kth word
    
    return min_count

In [6]:
ViTokenizer.tokenize("Chào mừng tân sinh viên đến với đại học Duy Tân")

'Chào_mừng tân_sinh_viên đến với đại_học Duy_Tân'

In [7]:
global lcode
print ("Making sentences as list...")
sents = []
with codecs.open('data/corpus/sample2.txt', 'r', 'utf-8-sig') as fin:
    while 1:
        line = fin.readline()
        line = re.sub(pattern, '', line)
        ## replace '[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]' by ''
        if not line: break

        words = ViTokenizer.tokenize(line)
        words = words.split()
        ## Split a sentence into list of words
        sents.append(words)
        
print ("Done making sentence list")

Making sentences as list...
Done making sentence list


In [8]:
sents

[['Cái',
  'kết',
  'đắng',
  'chát',
  'sau',
  '20',
  'năm',
  'đám_cưới',
  'là',
  'lời',
  'tuyên_bố',
  'của',
  'chồng',
  'tệ_bạc',
  'Giải_tán',
  'đi',
  'tôi',
  'lấy',
  'vợ',
  'khác'],
 ['Anh',
  'ta',
  'cười_phá',
  'lên',
  'như',
  'gã',
  'khùng',
  'rồi',
  'nói',
  'lại',
  'Con',
  'cô',
  'cô',
  'nuôi',
  'tất',
  'còn',
  'tài_sản',
  'thì',
  'làm',
  'gì',
  'có',
  'gì',
  'của',
  'cô',
  'mà',
  'chia'],
 ['Cô',
  'quên',
  'à',
  'mọi',
  'thứ',
  'đứng_tên',
  'tôi',
  'là',
  'tài_sản',
  'của',
  'riêng',
  'tôi'],
 ['Ngay',
  'ngày_mai',
  'mẹ',
  'con',
  'cô',
  'có_thể',
  'bước',
  'ra',
  'khỏi',
  'nhà',
  'tôi'],
 ['Tôi',
  'lấy',
  'chồng',
  'từ',
  'năm',
  '20',
  'tuổi',
  '24',
  'tuổi',
  'đã',
  'có',
  '2',
  'con',
  'mà',
  'công_việc',
  'của',
  '2',
  'vợ_chồng',
  'bấp_bênh',
  'kinh_tế',
  'thì',
  'khó_khăn'],
 ['Chồng',
  'bán',
  'cửa_hàng',
  'điện',
  'nước',
  'nhỏ',
  'ở',
  'chợ',
  'còn',
  'tôi',
  'đi',
  'bán',
  'h

In [12]:
# arguments setting 
lcode = "vi"
vector_size =300
window_size = 5
vocab_size = 30000
num_negative = 10
epochs = 15
print(get_min_count(sents))

83


In [13]:
print ("Making word vectors...")   

call_back = callback()

model = gensim.models.Word2Vec(sents, vector_size=300, min_count = 100,
                               negative=num_negative, 
                               window=window_size,
                               max_final_vocab=vocab_size,
                               callbacks=[call_back],
                               compute_loss=True,
                               sg = 1,
                               workers = 6,
                               epochs=epochs)

# Save vector to file
model.wv.save_word2vec_format('data/{}-{}-{}-{}-{}-loss.vec'.format(lcode, vector_size, window_size, vocab_size, num_negative), binary=False)

model.save('data/{}-{}-{}-{}-{}-loss.model'.format(lcode, vector_size, window_size, vocab_size, num_negative))
print ("Done.")

Making word vectors...
Loss after epoch 0: 86325480.0
Loss after epoch 1: 30434272.0
Loss after epoch 2: 17457976.0
Loss after epoch 3: 0.0
Loss after epoch 4: 0.0
Loss after epoch 5: 0.0
Loss after epoch 6: 0.0
Loss after epoch 7: 0.0
Loss after epoch 8: 0.0
Loss after epoch 9: 0.0
Loss after epoch 10: 0.0
Loss after epoch 11: 0.0
Loss after epoch 12: 0.0
Loss after epoch 13: 0.0
Loss after epoch 14: 0.0
Done.


In [40]:
model.wv.most_similar('Putin')

[('Tổng_thống', 0.649883508682251),
 ('Trump', 0.6207817196846008),
 ('Obama', 0.5937055945396423),
 ('Donald_Trump', 0.5491780638694763),
 ('tổng_thống', 0.5381444096565247),
 ('Bush', 0.5290942192077637),
 ('Nhà_Trắng', 0.4838489294052124),
 ('Melania', 0.48020875453948975),
 ('Lincoln', 0.47992777824401855),
 ('nghị_sĩ', 0.4753842055797577)]

In [35]:
vec = (model.wv['vua'] - model.wv['nam'] + model.wv['nữ'])
model.wv.most_similar([vec])

[('vua', 0.893063485622406),
 ('Vua', 0.7019286751747131),
 ('hoàng_hậu', 0.6391088366508484),
 ('Hoàng_hậu', 0.5814587473869324),
 ('Càn_Long', 0.5732150673866272),
 ('triều', 0.568006694316864),
 ('nhà_vua', 0.5524246692657471),
 ('thái_hậu', 0.5414510369300842),
 ('Thái_hậu', 0.5337126851081848),
 ('Tắc_Thiên', 0.5290496349334717)]